[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/openlayer-ai/examples-gallery/blob/main/text-classification/transformers/transformers.ipynb)

# <a id="top">Sentiment analysis using HuggingFace Transformers</a>

This notebook illustrates how transformer models can be uploaded to the Openlayer platform.

## <a id="toc">Table of contents</a>

1. [**Getting the data and training the model**](#1)
    - [Downloading the dataset](#download)
    - [Preparing the data](#prepare)
    - [Fine-tuning a transformer](#fine-tuning)
    

2. [**Using Openlayer's Python API**](#2)
    - [Instantiating the client](#client)
    - [Creating a project](#project)
    - [Uploading datasets](#dataset)
    - [Uploading models](#model)
        - [Shell models](#shell)
        - [Full models](#full-model)
    - [Committing and pushing to the platform](#commit)

In [ ]:
%%bash

if [ ! -e "requirements.txt" ]; then
    curl "https://raw.githubusercontent.com/openlayer-ai/examples-gallery/main/text-classification/transformers/requirements.txt" --output "requirements.txt"
fi

In [ ]:
!pip install -r requirements.txt

## <a id="1"> 1. Getting the data and training the model </a>

[Back to top](#top)

In this first part, we will get the dataset, pre-process it, split it into training and validation sets, and fine-tune a transformer. Feel free to skim through this section if you are already comfortable with how these steps look for a HuggingFace transformer.   

In [ ]:
import torch

from datasets import load_dataset
from scipy.special import softmax
from transformers import AutoTokenizer, AutoModelForSequenceClassification

### <a id="download">Downloading the dataset </a>


We will use the open-source [Yelp's Reviews](https://huggingface.co/datasets/yelp_review_full) dataset.

In [ ]:
dataset = load_dataset("yelp_review_full")

In [ ]:
dataset["train"][100]

In [ ]:
# For simplicity we'll only take 100 samples
training_set = dataset["train"].shuffle(seed=42).select(range(100))
validation_set = dataset["test"].shuffle(seed=42).select(range(100))

### <a id="prepare">Preparing the data</a>


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [ ]:
tokenized_training_set = training_set.map(tokenize_function, batched=True)
tokenized_validation_set = validation_set.map(tokenize_function, batched=True)

### <a id="model">Loading the model</a>

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-cased", 
    num_labels=5,
    ignore_mismatched_sizes=True
)

### <a id="fine-tuning">(Optional) Fine-tuning a transformer -- might take a long time to run</a>



We are going to use the `Trainer` class to fine-tune the transformer. It doesn't evaluate model performance during training by default, so the next few cells are taking care of that:

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

Now we can train the model:

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_training_set,
    eval_dataset=tokenized_validation_set,
)

In [ ]:
trainer.train()

## <a id="2"> 2. Using Openlayer's Python API</a>

[Back to top](#top)

Now it's time to upload the datasets and model to the Openlayer platform.

In [ ]:
!pip install openlayer

### <a id="client">Instantiating the client</a>

In [ ]:
import openlayer

client = openlayer.OpenlayerClient("YOUR_API_KEY_HERE")

### <a id="project">Creating a project on the platform</a>

In [ ]:
from openlayer.tasks import TaskType

project = client.create_or_load_project(
    name="Transformer Demo Project",
    task_type=TaskType.TextClassification,
    description="Project to Demo Transformers with Openlayer"
)

### <a id="dataset">Uploading datasets</a>

Before adding the datasets to a project, we need to do two things:
1. Enhance the dataset with additional columns to make it comprehensive, such as adding a column for labels and one for model predictions (if you're uploading a model as well).
2. Prepare a `dataset_config.yaml` file. This is a file that contains all the information needed by the Openlayer platform to utilize the dataset. It should include the column names, the class names, etc. For details on the fields of the `dataset_config.yaml` file, see the [API reference](https://reference.openlayer.com/reference/api/openlayer.OpenlayerClient.add_dataset.html#openlayer.OpenlayerClient.add_dataset).

Let's start by enhancing the datasets with the extra columns:

In [ ]:
train_df = training_set.to_pandas()
val_df = validation_set.to_pandas()

In [ ]:
from transformers import TextClassificationPipeline
from typing import List

pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, top_k=5)

def predicted_class_probabilities(text: str) -> List[float]:
    """From an input text, returns a list with the predicted
    class probabilities."""
    class_proba_dicts = pipe(text)
    
    class_proba_list = [0] * 5
    
    for item in class_proba_dicts:
        idx = int(item["label"].split("_")[1])
        class_proba_list[idx] = item["score"]
    
    return class_proba_list


In [ ]:
# Truncate the number of characters
train_df["text"] = train_df["text"].apply(lambda x: x[:1000])
val_df["text"] = val_df["text"].apply(lambda x: x[:1000])

In [ ]:
# Adding the column with the predictions (since we'll also upload a model later)
train_df["predictions"] = train_df["text"].apply(predicted_class_probabilities)
val_df["predictions"] = val_df["text"].apply(predicted_class_probabilities)

Now, we can prepare the `dataset_config.yaml` files for the training and validation sets.

In [ ]:
# Some variables that will go into the `dataset_config.yaml` file
column_names = list(train_df.columns)
class_names = ["1 star", "2 stars", "3 stars", "4 stars", "5 stars"]
label_column_name = "label"
prediction_scores_column_name = "predictions"
text_column_name = "text"

In [ ]:
import yaml 

# Note the camelCase for the dict's keys
training_dataset_config = {
    "classNames": class_names,
    "columnNames": column_names,
    "textColumnName": text_column_name,
    "label": "training",
    "labelColumnName": label_column_name,
    "predictionScoresColumnName": prediction_scores_column_name,
}

with open("training_dataset_config.yaml", "w") as dataset_config_file:
    yaml.dump(training_dataset_config, dataset_config_file, default_flow_style=False)

In [ ]:
import copy

validation_dataset_config = copy.deepcopy(training_dataset_config)

# In our case, the only field that changes is the `label`, from "training" -> "validation"
validation_dataset_config["label"] = "validation"

with open("validation_dataset_config.yaml", "w") as dataset_config_file:
    yaml.dump(validation_dataset_config, dataset_config_file, default_flow_style=False)

In [ ]:
# Training set
project.add_dataframe(
    dataset_df=train_df,
    dataset_config_file_path="training_dataset_config.yaml",
)

In [ ]:
# Validation set
project.add_dataframe(
    dataset_df=val_df,
    dataset_config_file_path="validation_dataset_config.yaml",
)

We can check that both datasets are now staged using the `project.status()` method. 

In [ ]:
project.status()

### <a id="model">Uploading models</a>

When it comes to uploading models to the Openlayer platform, there are two options:

- The first one is to upload a **shell model**. Shell models are the most straightforward way to get started. They are comprised of metadata and all of the analysis are done via its predictions (which are [uploaded with the datasets](#dataset)).
- The second one is to upload a **full model**, with artifacts. When a full model is uploaded, it becomes available in the platform and it becomes possible to perform what-if analysis, use all the explainability techniques available, and perform a series of robustness assessments with it. 

#### <a id="shell">Shell models</a>

To upload a shell model, we only need to define its name, the architecture type, and add some metadata that will be rendered in the platform to help us identify it. This information should be saved to a `model_config.yaml` file.

Let's create a `model_config.yaml` file for our model:

In [ ]:
import yaml

model_config = {
    "name": "Restaurant review model",
    "architectureType": "transformers",
    "metadata": {  # Can add anything here, as long as it is a dict
        "model_used": "bert-base-cased",
        "tokenizer_used": "bert-base-cased",
    },
    "classNames": class_names,
}

with open("model_config.yaml", "w") as model_config_file:
    yaml.dump(model_config, model_config_file, default_flow_style=False)

In [ ]:
project.add_model(
    model_config_file_path="model_config.yaml",
)

We can check that both datasets and model are staged using the `project.status()` method.

In [ ]:
project.status()

Since in this example, we're interested in uploading a full model, let's unstage the shell model:

In [ ]:
project.restore("model")

#### <a id="full-model"> Full models </a>

To upload a full model to Openlayer, you will need to create a model package, which is nothing more than a folder with all the necessary information to run inference with the model. The package should include the following:
1. A `requirements.txt` file listing the dependencies for the model.
2. Serialized model files, such as model weights, encoders, etc., in a format specific to the framework used for training (e.g. `.pkl` for sklearn, `.pb` for TensorFlow, and so on.)
3. A `prediction_interface.py` file that acts as a wrapper for the model and implements the `predict_proba` function. 

Other than the model package, a `model_config.yaml` file is needed, with information about the model to the Openlayer platform, such as the framework used, feature names, and categorical feature names.

Lets prepare the model package one piece at a time

In [ ]:
# Creating the model package folder (we'll call it `model_package`)
!mkdir model_package

**1. Adding the `requirements.txt` to the model package**

In [ ]:
!scp requirements.txt model_package

**2. Serializing the model and other objects needed**

In [ ]:
# Saving the pipeline (tokenizer and model)
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, top_k=5)

pipe.save_pretrained("model_package/pipeline")

**3. Writing the `prediction_interface.py` file**

In [ ]:
%%writefile model_package/prediction_interface.py
import numpy as np

from pathlib import Path
from typing import List
import pandas as pd
from transformers import pipeline

PACKAGE_PATH = Path(__file__).parent


class TransformerModel:
    def __init__(self):
        """This is where the serialized objects needed should
        be loaded as class attributes."""
        self.pipeline = pipeline(
            "text-classification", 
            str(PACKAGE_PATH) +  "/pipeline",
            top_k=5
        )
        
    def predict_proba(self, input_data_df: pd.DataFrame):
        """Makes predictions with the model. Returns the class probabilities."""
        text_column = input_data_df.columns[0]
        
        preds = input_data_df[text_column].apply(self._predict_row)

        return np.stack(preds.values)

    def _predict_row(self, text: str) -> List[float]:
        class_proba_dicts = self.pipeline(text)
    
        class_proba_list = [0] * 5

        for item in class_proba_dicts:
            idx = int(item["label"].split("_")[1])
            class_proba_list[idx] = item["score"]

        return class_proba_list
    
    
def load_model():
    """Function that returns the wrapped model object."""
    return TransformerModel()

**Creating the `model_config.yaml`**

In [ ]:
import yaml

model_config = {
    "name": "Restaurant review model",
    "architectureType": "transformers",
    "metadata": {  # Can add anything here, as long as it is a dict
        "model_used": "bert-base-cased",
        "tokenizer_used": "bert-base-cased",
    },
    "classNames": class_names,
}

with open("model_config.yaml", "w") as model_config_file:
    yaml.dump(model_config, model_config_file, default_flow_style=False)

Lets check that the model package contains everything needed:

In [ ]:
from openlayer.validators import model_validators

model_validator = model_validators.get_validator(
    task_type=TaskType.TextClassification,
    model_package_dir="model_package",
    model_config_file_path="model_config.yaml",
    sample_data = val_df[["text"]].iloc[:10, :]
)
model_validator.validate()

Now, we are ready to add the model:

In [ ]:
project.add_model(
    model_package_dir="model_package",
    model_config_file_path="model_config.yaml",
    sample_data=val_df[["text"]].iloc[:10, :]
)

We can check that both datasets and model are staged using the `project.status()` method.

In [ ]:
project.status()

### <a id="commit"> Committing and pushing to the platform </a>

Finally, we can commit the first project version to the platform. 

In [ ]:
project.commit("Initial commit!")

In [ ]:
project.status()

In [ ]:
project.push()